## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

## Read New CSV Into DF

In [2]:
path = 'MyQualtricsDownload/B2BSurvey.csv'

In [3]:
df = pd.read_csv(path, encoding = "ISO-8859-1")
df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_4,Referal_5,Referal_6
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,Click to write the question text,If you have a colleague that could answer the ...,If you have a colleague that could answer the ...,If you have a colleague that could answer the ...
1,"{""ImportId"":""startDate"",""timeZone"":""Z""}","{""ImportId"":""endDate"",""timeZone"":""Z""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""Z""}","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID3""}","{""ImportId"":""QID5""}","{""ImportId"":""QID6""}","{""ImportId"":""QID7""}","{""ImportId"":""QID8""}","{""ImportId"":""QID9""}","{""ImportId"":""QID10""}","{""ImportId"":""QID4_4""}","{""ImportId"":""QID4_5""}","{""ImportId"":""QID4_6""}"
2,2020-05-13 13:21:09,2020-05-13 13:21:49,0,74.96.76.163,100,39,1,2020-05-13 13:21:49,R_3hbTUPHEKabGVqN,NaN,...,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [4]:
#Drop the first two rows
df = df.drop([0,1])
df = df.iloc[:, np.r_[7,17:36]]
df.head()

,RecordedDate,Contact_Info_1,Contact_Info_2,Contact_Info_3,Contact_Info_8,Topic_1,Topic_2,Topic_3,Topic_1_Q1,Topic_1_Q2,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Referal_4,Referal_5,Referal_6
2,2020-05-13 13:21:49,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,1,1,NaN,2,3,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


In [5]:
#Rename contact and referal info
df = df.rename({'Contact_Info_1': 'Referer_Name', 'Contact_Info_2': 'Referer_Title',
                'Contact_Info_3':'Referer_Email','Contact_Info_8':'Company',
                'Referal_4':'Name','Referal_5':'Title','Referal_6':'Email',
                'Topic_1':'T1_Completed','Topic_2':'T2_Completed','Topic_3':'T3_Completed'}, axis=1)
df.head()

,RecordedDate,Referer_Name,Referer_Title,Referer_Email,Company,T1_Completed,T2_Completed,T3_Completed,Topic_1_Q1,Topic_1_Q2,Topic_1_Q3,Topic_2_Q1,Topic_2_Q2,Topic_2_Q3,Topic_3_Q1,Topic_3_Q2,Topic_3_Q3,Name,Title,Email
2,2020-05-13 13:21:49,Monty Mole,Mole,monty_mole@mailinator.com,Nintendo,1,1,NaN,2,3,1,2,1,1,NaN,NaN,NaN,King Boo,Ghost,kingboo32@mailinator.com


## Clean Up New Contacts From Referals

In [6]:
NewContacts = df[['Name','Title','Email',
                  'Company','Referer_Name',
                  'Referer_Title','Referer_Email',
                  'T1_Completed','T2_Completed','T3_Completed']]
NewContacts.head()

,Name,Title,Email,Company,Referer_Name,Referer_Title,Referer_Email,T1_Completed,T2_Completed,T3_Completed
2,King Boo,Ghost,kingboo32@mailinator.com,Nintendo,Monty Mole,Mole,monty_mole@mailinator.com,1,1,NaN


In [7]:
#What if the same person gets recommended? merge rows
aggregation_functions = {'Company':'first', 'Name':'first', 'Email':'first',
                         'Referer_Name':'first', 'Referer_Title':'first', 'Referer_Email':'first',
                         'T1_Completed':'sum', 'T2_Completed':'sum', 'T3_Completed':'sum'}
NewContacts = NewContacts.groupby(NewContacts['Email']).aggregate(aggregation_functions)

del NewContacts['Email']
NewContacts = NewContacts.reset_index()

In [8]:
#Make columns numeric
cols = ['T1_Completed','T2_Completed','T3_Completed']
NewContacts[cols] = NewContacts[cols].apply(pd.to_numeric, errors='coerce', axis=1)
NewContacts.head()

,Email,Company,Name,Referer_Name,Referer_Title,Referer_Email,T1_Completed,T2_Completed,T3_Completed
0,kingboo32@mailinator.com,Nintendo,King Boo,Monty Mole,Mole,monty_mole@mailinator.com,1,1,0


In [9]:
#if completed values are not 0, make them 1
NewContacts.loc[NewContacts['T1_Completed'] > 0, 'T1_Completed'] = 1
NewContacts.loc[NewContacts['T2_Completed'] > 0, 'T2_Completed'] = 1
NewContacts.loc[NewContacts['T3_Completed'] > 0, 'T3_Completed'] = 1
NewContacts

,Email,Company,Name,Referer_Name,Referer_Title,Referer_Email,T1_Completed,T2_Completed,T3_Completed
0,kingboo32@mailinator.com,Nintendo,King Boo,Monty Mole,Mole,monty_mole@mailinator.com,1,1,0


## Create Custom Links

In [10]:
survey_link = '<<insert anonymous survey link>>' + '?'

In [11]:
#Add embedded data to survey link
NewContacts['SurveyLink'] = survey_link + 'Company=' + NewContacts['Company'].astype(str) + '&T1_Completed=' + NewContacts['T1_Completed'].astype(str) + '&T2_Completed=' + NewContacts['T2_Completed'].astype(str) + '&T3_Completed=' + NewContacts['T3_Completed'].astype(str)
print(NewContacts['SurveyLink'][0])

<<insert anonymous survey link>>?Company=Nintendo&T1_Completed=1&T2_Completed=1&T3_Completed=0


## Send Referal Email

This blog will show you how to configure a gmail account to send emails in a python script: https://realpython.com/python-send-email/

In [12]:
sender_email = "<<insert sender email address>>" 
password = "<<insert sender email password>>"

In [15]:
for index, row in NewContacts.iterrows():

    receiver_email = row['Email']

    message = MIMEMultipart("alternative")
    message["Subject"] = "multipart test"
    message["From"] = sender_email
    message["To"] = receiver_email

    # Create the plain-text and HTML version of your message
    text = """\
    Hi {first_name},
    
    My name is ______ and I work at _____. We are collecting information on ____ to ____.
    
    Your colleague {referer_name} has already provided some of this information and has listed you as someone who could help fill in the remaining information. 

    Please follow the link to this survey to complete the remaining information: {survey_link}
    
    Thank you for supporting this effort!""".format(first_name = row['Name'].split(' ')[0],
                                                    referer_name = row['Referer_Name'],
                                                    survey_link = row['SurveyLink'])
    
    html = """\
    <html>
      <body>
        <p>Hi {first_name},
           <br>My name is ______ and I work at _____. We are collecting information on ____ to ____.
           <br>Your colleague {referer_name} has already provided some of this information and has listed you as someone who could help fill in the remaining information.
           <br>Please take <a href={survey_link}>this survey</a> to complete the remaining information.
           <br>Thank you for supporting this effort!
        </p>
      </body>
    </html>
    """.format(first_name = row['Name'].split(' ')[0],
               referer_name = row['Referer_Name'],
               survey_link = row['SurveyLink'])

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    message.attach(part2)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string()
        )

    Hi King,
    
    My name is ______ and I work at _____. We are collecting information on ____ to ____.
    
    Your colleague Monty Mole has already provided some of this information and has listed you as someone who could help fill in the remaining information. 

    Please follow the link to this survey to complete the remaining information: <<insert anonymous survey link>>?Company=Nintendo&T1_Completed=1&T2_Completed=1&T3_Completed=0
    
    Thank you for supporting this effort!
    <html>
      <body>
        <p>Hi King,<br>
           My name is ______ and I work at _____. We are collecting information on ____ to ____.<br>
           Your colleague Monty Mole has already provided some of this information and has listed you as someone who could help fill in the remaining information.<br>
           Please take <a href=<<insert anonymous survey link>>?Company=Nintendo&T1_Completed=1&T2_Completed=1&T3_Completed=0>this survey</a> to complete the remaining information.<br>
     